# TEMPLATE JAWABAN TUGAS LANGUAGE MODELING

### Import Library

In [1]:
import nltk
import xlrd
import operator
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Dataset (Training dan Testing)

In [12]:
book = xlrd.open_workbook('dataTrainTitle.xlsx')
sheet = book.sheet_by_name('Sheet1')
dataTrain = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]
#add dummy-ish data for OOV, the OOV will be converted to _____
#just 4 basic word classes, noun, verbs, adjectives, adverb
dataTrain.append(['the _____ has _____ a new car that will drive arond the city _____']) 
#the <noun> has <verb> a new car that will drive around the city <adverb>
dataTrain.append(['the _____ is one _____ car'])
#the <noun> is one <adjective> car
print(dataTrain)

[["It'll Be Years Before The New 'R36' Nissan GT-R Arrives"], ['A New, Potentially Less Ugly Nissan Juke Is On The Way'], ['The 2020 Nissan GT-R Nismo Is Almost 600LT Money'], ['the _____ has _____ a new car that will drive arond the city _____'], ['the _____ is one _____ car']]


### Bangun Model Unigram

In [13]:
def buildUnigramModel(trainingSentences):
    wordCount = {}
    count = 0
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0])
        for token in tokens:
            tokenLower = token.lower()
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
            count += 1
    unigramModel = wordCount
    #count prob
    for word in unigramModel:
        unigramModel[word] = unigramModel[word] / count
    return unigramModel
unigramModel = buildUnigramModel(dataTrain)
print(unigramModel)

{'it': 0.018867924528301886, "'ll": 0.018867924528301886, 'be': 0.018867924528301886, 'years': 0.018867924528301886, 'before': 0.018867924528301886, 'the': 0.11320754716981132, 'new': 0.05660377358490566, "'r36": 0.018867924528301886, "'": 0.018867924528301886, 'nissan': 0.05660377358490566, 'gt-r': 0.03773584905660377, 'arrives': 0.018867924528301886, 'a': 0.03773584905660377, ',': 0.018867924528301886, 'potentially': 0.018867924528301886, 'less': 0.018867924528301886, 'ugly': 0.018867924528301886, 'juke': 0.018867924528301886, 'is': 0.05660377358490566, 'on': 0.018867924528301886, 'way': 0.018867924528301886, '2020': 0.018867924528301886, 'nismo': 0.018867924528301886, 'almost': 0.018867924528301886, '600lt': 0.018867924528301886, 'money': 0.018867924528301886, '_____': 0.09433962264150944, 'has': 0.018867924528301886, 'car': 0.03773584905660377, 'that': 0.018867924528301886, 'will': 0.018867924528301886, 'drive': 0.018867924528301886, 'arond': 0.018867924528301886, 'city': 0.0188679

### Bangun Model Bigram

In [4]:
def buildBigramModel(trainingSentences):
    wordPairCount = {}
    wordCount = {}
    wordCount['<s>'] = 0
    wordCount['</s>'] = 0
    wordPairCount['<s>'] = {}
    #count word
    for sentence in trainingSentences:
        tokens = nltk.word_tokenize(sentence[0])
        counter = 0
        tokenBefore = ''
        wordCount['<s>'] += 1
        wordCount['</s>'] += 1
        for token in tokens:
            tokenLower = token.lower()
            counter += 1
            if tokenLower not in wordPairCount:
                wordPairCount[tokenLower] = {}
            if counter == 1:
                if tokenLower in wordPairCount['<s>']:
                    wordPairCount['<s>'][tokenLower] += 1
                else:
                    wordPairCount['<s>'][tokenLower] = 1
            elif counter == len(tokens):
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
                if '</s>' in wordPairCount[tokenLower]:
                    wordPairCount[tokenLower]['</s>'] += 1
                else:
                    wordPairCount[tokenLower]['</s>'] = 1
            else:
                if tokenLower in wordPairCount[tokenBefore]:
                    wordPairCount[tokenBefore][tokenLower] += 1
                else:
                    wordPairCount[tokenBefore][tokenLower] = 1
            tokenBefore = tokenLower
            if tokenLower in wordCount:
                wordCount[tokenLower] += 1
            else:
                wordCount[tokenLower] = 1
    bigramModel = {}
    vee = len(wordCount) #for smoothing
    #smoothing (adding one)
    for wordBefore in wordPairCount.keys():
        for theWord in wordCount.keys():
            if theWord != '<s>':
                if theWord not in wordPairCount[wordBefore]:
                    wordPairCount[wordBefore][theWord] = 1
                else:
                    wordPairCount[wordBefore][theWord] += 1
                #bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    #smoothing (adding vee)
    for theWord in wordCount.keys():
        wordCount[theWord] += vee
    #count prob
    for wordBefore in wordPairCount.keys():
        bigramModel[wordBefore] = {}
        for theWord in wordCount.keys():
            if theWord != '<s>':
                bigramModel[wordBefore][theWord] = wordPairCount[wordBefore][theWord] / wordCount[theWord]
    return bigramModel
bigramModel = buildBigramModel(dataTrain)

### Bangun Fungsi Kata Berikutnya Dengan Probability Tertinggi

In [5]:
#Diberikan sebuah kata, cari kata berikutnya yang memiliki probability tertinggi berdasarkan model bigram
def nextBestWord(bigramModel, currentWord):
    currentWordLow = currentWord.lower()
    if currentWordLow not in bigramModel:
        nextWord = "OOV"
    else:
        nextWord = max(bigramModel[currentWordLow].items(), key=operator.itemgetter(1))[0]
    return nextWord
#Lakukan percobaan dengan memberikan sebuah kata, lalu print kata berikutnya
print(nextBestWord(bigramModel,"Nissan"))

gt-r


### Hitung Perplexity Unigram dan Bigram terhadap Data Testing

In [ ]:
def countPerplexity(..........., testSentences):
    

    
    return perplexity

### Bangun Fungsi Generate Kalimat

In [ ]:
def generateSentence(......, length):
    
    
    
    return sentence